In [5]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7000


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Meshcat: http://localhost:7000


In [13]:
# simulate_two.py
import os
import numpy as np
from pydrake.all import StartMeshcat
from pydrake.multibody.tree import RevoluteJoint
from rocky.build import build_robot_diagram_two

# --- resolve URDF path ---
urdf = "rocky.urdf"  # URDF should have a body/frame named "wall"

# --- build diagram with two robot instances ---
bundle = build_robot_diagram_two(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., -9.81),
    meshcat=meshcat,
)
plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

# --- helpers: control per model instance ---
def get_revolute_joints_for_instance(plant, instance):
    joints = [plant.get_joint(j_idx) for j_idx in plant.GetJointIndices(instance)]
    return [j for j in joints if isinstance(j, RevoluteJoint)]

def set_angles_for_instance(plant, ctx, instance, angles_rad):
    revs = get_revolute_joints_for_instance(plant, instance)
    if len(angles_rad) != len(revs):
        raise ValueError(f"Expected {len(revs)} angles, got {len(angles_rad)}")
    for j, q in zip(revs, angles_rad):
        j.set_angle(ctx, float(q))

def set_zero_torques(plant, ctx):
    # zero all actuators
    tau = np.zeros(plant.num_actuators())
    plant.get_actuation_input_port().FixValue(ctx, tau)

# --- choose simple 2-DoF targets (edit as needed) ---
ally_angles_deg  = [-45.0, -60.0]  # shoulder, elbow (example)
enemy_angles_deg = [  50.0,   0.0]

ally_angles  = np.deg2rad(ally_angles_deg)
enemy_angles = np.deg2rad(enemy_angles_deg)

# --- set each arm independently ---
set_angles_for_instance(plant, ctx, ally,  ally_angles)
set_angles_for_instance(plant, ctx, enemy, enemy_angles)

# passive sim
set_zero_torques(plant, ctx)
sim.AdvanceTo(1.0)
print("Done.")


Done.
